In [ ]:
from mmseg.registry import MODELS
import torch
import torch.nn as nn
from mmcv.cnn import ConvModule

from mmseg.registry import MODELS
from mmseg.models.decode_heads.decode_head import BaseDecodeHead


from mmseg.models.utils import resize,UpConvBlock, Upsample
from mmseg.models.decode_heads.psp_head import PPM

import torch.utils.checkpoint as cp
from mmcv.cnn import ConvModule, build_activation_layer, build_norm_layer
from mmengine.model import BaseModule
from mmengine.utils.dl_utils.parrots_wrapper import _BatchNorm

class BasicConvBlock(nn.Module):
    """Basic convolutional block for UNet.

    This module consists of several plain convolutional layers.

    Args:
        in_channels (int): Number of input channels.
        out_channels (int): Number of output channels.
        num_convs (int): Number of convolutional layers. Default: 2.
        stride (int): Whether use stride convolution to downsample
            the input feature map. If stride=2, it only uses stride convolution
            in the first convolutional layer to downsample the input feature
            map. Options are 1 or 2. Default: 1.
        dilation (int): Whether use dilated convolution to expand the
            receptive field. Set dilation rate of each convolutional layer and
            the dilation rate of the first convolutional layer is always 1.
            Default: 1.
        with_cp (bool): Use checkpoint or not. Using checkpoint will save some
            memory while slowing down the training speed. Default: False.
        conv_cfg (dict | None): Config dict for convolution layer.
            Default: None.
        norm_cfg (dict | None): Config dict for normalization layer.
            Default: dict(type='BN').
        act_cfg (dict | None): Config dict for activation layer in ConvModule.
            Default: dict(type='ReLU').
        dcn (bool): Use deformable convolution in convolutional layer or not.
            Default: None.
        plugins (dict): plugins for convolutional layers. Default: None.
    """

    def __init__(self,
                 in_channels,
                 out_channels,
                 num_convs=2,
                 stride=1,
                 dilation=1,
                 with_cp=False,
                 conv_cfg=None,
                 norm_cfg=dict(type='BN'),
                 act_cfg=dict(type='ReLU'),
                 dcn=None,
                 plugins=None):
        super().__init__()
        assert dcn is None, 'Not implemented yet.'
        assert plugins is None, 'Not implemented yet.'

        self.with_cp = with_cp
        convs = []
        for i in range(num_convs):
            convs.append(
                ConvModule(
                    in_channels=in_channels if i == 0 else out_channels,
                    out_channels=out_channels,
                    kernel_size=3,
                    stride=stride if i == 0 else 1,
                    dilation=1 if i == 0 else dilation,
                    padding=1 if i == 0 else dilation,
                    conv_cfg=conv_cfg,
                    norm_cfg=norm_cfg,
                    act_cfg=act_cfg))

        self.convs = nn.Sequential(*convs)

    def forward(self, x):
        """Forward function."""

        if self.with_cp and x.requires_grad:
            out = cp.checkpoint(self.convs, x)
        else:
            out = self.convs(x)
        return out


@MODELS.register_module()
class UnetDecoder(BaseDecodeHead):
    """Unified Perceptual Parsing for Scene Understanding.

    This head is the implementation of `UPerNet
    <https://arxiv.org/abs/1807.10221>`_.

    Args:
        pool_scales (tuple[int]): Pooling scales used in Pooling Pyramid
            Module applied on the last feature. Default: (1, 2, 3, 6).
    """

    def __init__(self, num_stages = 3, **kwargs):
        super().__init__(input_transform='multiple_select', **kwargs)
        # PSP Module
        upsample_cfg=dict(type='InterpConv')
        self.decoder = nn.ModuleList()
        self.num_stages = num_stages
#         self.encoder = []
#         self.encoder.append(BasicConvBlock(
#                     in_channels=self.in_channels[-1],
#                     out_channels=self.in_channels[-1] ** 2,
#                     num_convs=2,
#                     stride=1,
#                     dilation=1,
#                     with_cp=False,
#                     conv_cfg=None,
#                     norm_cfg=dict(type='BN'),
#                     act_cfg=dict(type='ReLU'),
#                     dcn=None,
#                     plugins=None))
        for i in range(num_stages - 1):
            #print(self.in_channels[num_stages - i-1])
            #print(self.in_channels[num_stages -i-2])
            self.decoder.append(
                    UpConvBlock(
                        conv_block=BasicConvBlock,
                        in_channels=self.in_channels[num_stages - i-1],
                        skip_channels=self.in_channels[num_stages -i-2],
                        out_channels=self.in_channels[num_stages -i-2],
                        num_convs=2,
                        stride=1,
                        dilation=1,
                        with_cp=False,
                        conv_cfg=None,
                        norm_cfg=dict(type='BN'),
                        act_cfg=dict(type='ReLU'),
                        upsample_cfg=upsample_cfg,
                        dcn=None,
                        plugins=None))



    def _forward_feature(self, inputs):
        """Forward function for feature maps before classifying each pixel with
        ``self.cls_seg`` fc.

        Args:
            inputs (list[Tensor]): List of multi-level img features.

        Returns:
            feats (Tensor): A tensor of shape (batch_size, self.channels,
                H, W) which is feature map for last layer of decoder head.
        """
        inputs = self._transform_inputs(inputs)

#         print(len(inputs))
#         for inp in inputs:
#             print(inp.shape)
        x = inputs[-1]
        for i in range(self.num_stages - 1):
            x = self.decoder[i](inputs[-2-i],x)
            #print(x.shape)
        #bottleneck = self.bottleneck(inputs[-1])
        #print(bottleneck.shape)
        return x

    def forward(self, inputs):
        """Forward function."""
        output = self._forward_feature(inputs)
        output = self.cls_seg(output)
        return output

In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from mmwrapper.src.api import get_runner

C:\Users\philmarq\miniconda3\envs\mmtest\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#runner = get_runner(r"C:\Users\philmarq\source\repos\MMWrapperRepo\mmwrapper\src\configs\object_detection\object_detection.yaml")
#runner = get_runner(r"C:\Users\philmarq\source\repos\MMWrapperRepo\mmwrapper\src\configs\segmentation\segmentation.yaml")
#runner = get_runner(r"C:\Users\philmarq\source\repos\MMWrapperRepo\mmwrapper\src\configs\instance_segmentation\instance_seg.yaml")
runner = get_runner(r"C:\Users\philmarq\source\repos\MMWrapperRepo\mmwrapper\src\configs\classification\classification.yaml")

12/14 09:10:21 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: win32
    Python: 3.9.16 (main, May 17 2023, 17:49:16) [MSC v.1916 64 bit (AMD64)]
    CUDA available: True
    numpy_random_seed: 1088709692
    GPU 0: NVIDIA GeForce RTX 3090 Ti
    CUDA_HOME: None
    MSVC: n/a, reason: fileno
    PyTorch: 2.0.1+cu117
    PyTorch compiling details: PyTorch built with:
  - C++ Version: 199711
  - MSVC 193431937
  - Intel(R) Math Kernel Library Version 2020.0.2 Product Build 20200624 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 2019
  - LAPACK is enabled (usually provided by MKL)
  - CPU capability usage: AVX2
  - CUDA Runtime 11.7
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arch=compute_70,code=s

In [ ]:
runner.train()

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmdetection/v3.0/mask2former/mask2former_swin-t-p4-w7-224_8xb2-lsj-50e_coco/mask2former_swin-t-p4-w7-224_8xb2-lsj-50e_coco_20220508_091649-01b0f990.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: panoptic_head.pixel_decoder.input_convs.0.conv.weight, panoptic_head.pixel_decoder.input_convs.0.conv.bias, panoptic_head.pixel_decoder.input_convs.0.gn.weight, panoptic_head.pixel_decoder.input_convs.0.gn.bias, panoptic_head.pixel_decoder.input_convs.1.conv.weight, panoptic_head.pixel_decoder.input_convs.1.conv.bias, panoptic_head.pixel_decoder.input_convs.1.gn.weight, panoptic_head.pixel_decoder.input_convs.1.gn.bias, panoptic_head.pixel_decoder.input_convs.2.conv.weight, panoptic_head.pixel_decoder.input_convs.2.conv.bias, panoptic_head.pixel_decoder.input_convs.2.gn.weight, panoptic_head.pixel_decoder.input_convs.2.gn.bias, panoptic_head.pixel_decoder.encoder.layers.0.s

12/14 09:10:23 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
12/14 09:10:23 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.
12/14 09:10:23 - mmengine - INFO - Checkpoints will be saved to C:\Users\philmarq\source\repos\MMWrapperRepo\testclassification.
12/14 09:10:28 - mmengine - INFO - Epoch(train)   [1][ 50/164]  lr: 1.0000e-01  eta: 1:24:41  time: 0.1034  data_time: 0.0276  memory: 544  loss: 5.1251
12/14 09:10:31 - mmengine - INFO - Epoch(train)   [1][100/164]  lr: 9.9999e-02  eta: 1:03:21  time: 0.0515  data_time: 0.0222  memory: 544  loss: 1.1060
12/14 09:10:33 - mmengine - INFO - Epoch(train)   [1][150/164]  lr: 9.9998e-02  eta: 0:54:34  time: 0.0454  data_time: 0.0182  memory: 544  loss: 0.8201
12/14 09:10:34 - mmengine - INFO - Exp name: resnet_20231214_091019
12/14 09:10:34 - mmengine - INFO - Sav

12/14 09:12:16 - mmengine - INFO - Epoch(train)   [7][ 50/164]  lr: 9.9891e-02  eta: 0:47:32  time: 0.0594  data_time: 0.0307  memory: 544  loss: 0.7328
12/14 09:12:19 - mmengine - INFO - Epoch(train)   [7][100/164]  lr: 9.9880e-02  eta: 0:47:26  time: 0.0576  data_time: 0.0315  memory: 544  loss: 0.7109
12/14 09:12:22 - mmengine - INFO - Epoch(train)   [7][150/164]  lr: 9.9869e-02  eta: 0:47:15  time: 0.0555  data_time: 0.0271  memory: 544  loss: 0.6981
12/14 09:12:22 - mmengine - INFO - Exp name: resnet_20231214_091019
12/14 09:12:22 - mmengine - INFO - Saving checkpoint at 7 epochs
12/14 09:12:25 - mmengine - INFO - Epoch(val)   [7][ 50/164]    eta: 0:00:04  time: 0.0426  data_time: 0.0311  memory: 544  
12/14 09:12:27 - mmengine - INFO - Epoch(val)   [7][100/164]    eta: 0:00:02  time: 0.0441  data_time: 0.0282  memory: 339  
12/14 09:12:30 - mmengine - INFO - Epoch(val)   [7][150/164]    eta: 0:00:00  time: 0.0457  data_time: 0.0277  memory: 339  
12/14 09:12:30 - mmengine - INFO 

12/14 09:14:13 - mmengine - INFO - Epoch(val)  [13][150/164]    eta: 0:00:00  time: 0.0447  data_time: 0.0272  memory: 339  
12/14 09:14:14 - mmengine - INFO - Epoch(val) [13][164/164]    accuracy/top1: 59.4512  data_time: 0.0283  time: 0.0432
12/14 09:14:17 - mmengine - INFO - Epoch(train)  [14][ 50/164]  lr: 9.9516e-02  eta: 0:45:29  time: 0.0594  data_time: 0.0298  memory: 544  loss: 0.7954
12/14 09:14:20 - mmengine - INFO - Epoch(train)  [14][100/164]  lr: 9.9494e-02  eta: 0:45:25  time: 0.0567  data_time: 0.0303  memory: 544  loss: 0.6923
12/14 09:14:22 - mmengine - INFO - Epoch(train)  [14][150/164]  lr: 9.9471e-02  eta: 0:45:21  time: 0.0565  data_time: 0.0288  memory: 544  loss: 0.7324
12/14 09:14:23 - mmengine - INFO - Exp name: resnet_20231214_091019
12/14 09:14:23 - mmengine - INFO - Saving checkpoint at 14 epochs
12/14 09:14:26 - mmengine - INFO - Epoch(val)  [14][ 50/164]    eta: 0:00:04  time: 0.0400  data_time: 0.0290  memory: 544  
12/14 09:14:28 - mmengine - INFO - Epo

12/14 09:16:09 - mmengine - INFO - Epoch(val)  [20][ 50/164]    eta: 0:00:04  time: 0.0417  data_time: 0.0288  memory: 544  
12/14 09:16:11 - mmengine - INFO - Epoch(val)  [20][100/164]    eta: 0:00:02  time: 0.0429  data_time: 0.0268  memory: 339  
12/14 09:16:13 - mmengine - INFO - Epoch(val)  [20][150/164]    eta: 0:00:00  time: 0.0450  data_time: 0.0286  memory: 339  
12/14 09:16:14 - mmengine - INFO - Epoch(val) [20][164/164]    accuracy/top1: 40.5488  data_time: 0.0283  time: 0.0435
12/14 09:16:17 - mmengine - INFO - Epoch(train)  [21][ 50/164]  lr: 9.8875e-02  eta: 0:44:05  time: 0.0600  data_time: 0.0316  memory: 544  loss: 0.8316
12/14 09:16:20 - mmengine - INFO - Epoch(train)  [21][100/164]  lr: 9.8841e-02  eta: 0:44:00  time: 0.0544  data_time: 0.0264  memory: 544  loss: 0.7441
12/14 09:16:23 - mmengine - INFO - Epoch(train)  [21][150/164]  lr: 9.8806e-02  eta: 0:43:55  time: 0.0552  data_time: 0.0269  memory: 544  loss: 0.7138
12/14 09:16:23 - mmengine - INFO - Exp name: re

...

12/14 09:39:34 - mmengine - INFO - Epoch(train) [102][100/164]  lr: 7.4269e-02  eta: 0:30:59  time: 0.0556  data_time: 0.0291  memory: 544  loss: 0.7122
12/14 09:39:37 - mmengine - INFO - Epoch(train) [102][150/164]  lr: 7.4130e-02  eta: 0:30:56  time: 0.0566  data_time: 0.0293  memory: 544  loss: 0.7128
12/14 09:39:38 - mmengine - INFO - Exp name: resnet_20231214_091019
12/14 09:39:38 - mmengine - INFO - Saving checkpoint at 102 epochs
12/14 09:39:40 - mmengine - INFO - Epoch(val) [102][ 50/164]    eta: 0:00:04  time: 0.0410  data_time: 0.0290  memory: 544  
12/14 09:39:43 - mmengine - INFO - Epoch(val) [102][100/164]    eta: 0:00:02  time: 0.0432  data_time: 0.0270  memory: 339  
12/14 09:39:45 - mmengine - INFO - Epoch(val) [102][150/164]    eta: 0:00:00  time: 0.0440  data_time: 0.0279  memory: 339  
12/14 09:39:45 - mmengine - INFO - Epoch(val) [102][164/164]    accuracy/top1: 40.5488  data_time: 0.0283  time: 0.0429
12/14 09:39:48 - mmengine - INFO - Epoch(train) [103][ 50/164]  

12/14 09:41:29 - mmengine - INFO - Epoch(val) [108][164/164]    accuracy/top1: 59.4512  data_time: 0.0281  time: 0.0428
12/14 09:41:32 - mmengine - INFO - Epoch(train) [109][ 50/164]  lr: 7.1147e-02  eta: 0:29:56  time: 0.0581  data_time: 0.0295  memory: 544  loss: 0.7797
12/14 09:41:35 - mmengine - INFO - Epoch(train) [109][100/164]  lr: 7.1003e-02  eta: 0:29:53  time: 0.0542  data_time: 0.0262  memory: 544  loss: 0.7285
12/14 09:41:38 - mmengine - INFO - Epoch(train) [109][150/164]  lr: 7.0858e-02  eta: 0:29:50  time: 0.0591  data_time: 0.0317  memory: 544  loss: 0.6783
12/14 09:41:38 - mmengine - INFO - Exp name: resnet_20231214_091019
12/14 09:41:38 - mmengine - INFO - Saving checkpoint at 109 epochs
12/14 09:41:41 - mmengine - INFO - Epoch(val) [109][ 50/164]    eta: 0:00:04  time: 0.0416  data_time: 0.0302  memory: 544  
12/14 09:41:43 - mmengine - INFO - Epoch(val) [109][100/164]    eta: 0:00:02  time: 0.0431  data_time: 0.0269  memory: 339  
12/14 09:41:46 - mmengine - INFO - E

12/14 09:43:27 - mmengine - INFO - Epoch(val) [115][100/164]    eta: 0:00:02  time: 0.0432  data_time: 0.0279  memory: 339  
12/14 09:43:30 - mmengine - INFO - Epoch(val) [115][150/164]    eta: 0:00:00  time: 0.0448  data_time: 0.0289  memory: 339  
12/14 09:43:30 - mmengine - INFO - Epoch(val) [115][164/164]    accuracy/top1: 40.5488  data_time: 0.0289  time: 0.0434
12/14 09:43:33 - mmengine - INFO - Epoch(train) [116][ 50/164]  lr: 6.7772e-02  eta: 0:28:51  time: 0.0582  data_time: 0.0273  memory: 544  loss: 0.7825
12/14 09:43:36 - mmengine - INFO - Epoch(train) [116][100/164]  lr: 6.7623e-02  eta: 0:28:48  time: 0.0562  data_time: 0.0280  memory: 544  loss: 0.7382
12/14 09:43:38 - mmengine - INFO - Exp name: resnet_20231214_091019
12/14 09:43:39 - mmengine - INFO - Epoch(train) [116][150/164]  lr: 6.7473e-02  eta: 0:28:45  time: 0.0569  data_time: 0.0304  memory: 544  loss: 0.7376
12/14 09:43:40 - mmengine - INFO - Exp name: resnet_20231214_091019
12/14 09:43:40 - mmengine - INFO - 

12/14 09:45:23 - mmengine - INFO - Exp name: resnet_20231214_091019
12/14 09:45:23 - mmengine - INFO - Exp name: resnet_20231214_091019
12/14 09:45:23 - mmengine - INFO - Saving checkpoint at 122 epochs
12/14 09:45:26 - mmengine - INFO - Epoch(val) [122][ 50/164]    eta: 0:00:04  time: 0.0402  data_time: 0.0301  memory: 544  
12/14 09:45:28 - mmengine - INFO - Epoch(val) [122][100/164]    eta: 0:00:02  time: 0.0430  data_time: 0.0268  memory: 339  
12/14 09:45:30 - mmengine - INFO - Epoch(val) [122][150/164]    eta: 0:00:00  time: 0.0442  data_time: 0.0275  memory: 339  
12/14 09:45:31 - mmengine - INFO - Epoch(val) [122][164/164]    accuracy/top1: 59.4512  data_time: 0.0285  time: 0.0432
12/14 09:45:34 - mmengine - INFO - Epoch(train) [123][ 50/164]  lr: 6.4302e-02  eta: 0:27:45  time: 0.0582  data_time: 0.0296  memory: 544  loss: 0.7129
12/14 09:45:37 - mmengine - INFO - Epoch(train) [123][100/164]  lr: 6.4149e-02  eta: 0:27:42  time: 0.0548  data_time: 0.0291  memory: 544  loss: 0.7

12/14 09:47:21 - mmengine - INFO - Epoch(train) [129][100/164]  lr: 6.1110e-02  eta: 0:26:46  time: 0.0554  data_time: 0.0302  memory: 544  loss: 0.6860
12/14 09:47:24 - mmengine - INFO - Epoch(train) [129][150/164]  lr: 6.0954e-02  eta: 0:26:43  time: 0.0609  data_time: 0.0319  memory: 544  loss: 0.7180
12/14 09:47:24 - mmengine - INFO - Exp name: resnet_20231214_091019
12/14 09:47:24 - mmengine - INFO - Saving checkpoint at 129 epochs
12/14 09:47:28 - mmengine - INFO - Epoch(val) [129][ 50/164]    eta: 0:00:05  time: 0.0461  data_time: 0.0313  memory: 544  
12/14 09:47:30 - mmengine - INFO - Epoch(val) [129][100/164]    eta: 0:00:03  time: 0.0548  data_time: 0.0361  memory: 339  
12/14 09:47:33 - mmengine - INFO - Epoch(val) [129][150/164]    eta: 0:00:00  time: 0.0475  data_time: 0.0309  memory: 339  
12/14 09:47:33 - mmengine - INFO - Epoch(val) [129][164/164]    accuracy/top1: 59.4512  data_time: 0.0327  time: 0.0493
12/14 09:47:36 - mmengine - INFO - Epoch(train) [130][ 50/164]  

# Instance seg inference

In [ ]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#import pydensecrf.densecrf as dcrf

from mmdet.apis import init_detector, inference_detector

import numpy as np
import cv2



config_file = r"C:\Users\philmarq\source\repos\MMWrapperRepo\megdata\swin_s.py"
checkpoint_file = r"C:\Users\philmarq\source\repos\MMWrapperRepo\megdata\best_coco_bbox_mAP_epoch_55.pth"

device = 'cuda:0'

model = init_detector(config_file, checkpoint_file, device=device)

# Load image


colors = [(255,0,0), (125,0,0), (0,255,0), (50, 255,0), (0,25,255), (0, 125, 0)]
root = r"C:\HSA\HSA-Data\projects\2cc375cc-4067-40f6-a23d-387ec7422611\dataset\images"
for file in os.listdir(root):
    img = os.path.join(root, file)
    result = inference_detector(model, img)
    
    

    color = (255,0,0)
    image = cv2.imread(img)
    #image[image>0] = 0
    masks = result.pred_instances.masks.cpu().numpy()
    scores = result.pred_instances.scores.cpu().numpy()
    labels = result.pred_instances.labels.cpu().numpy()
    bboxes = result.pred_instances.bboxes.cpu().numpy()
    thresh = 0.75
    
    for mask, score,label, bbox in zip(masks, scores,  labels, bboxes):
        if score > thresh:
            if label == 0:  # For label == 0, draw contours
                mask = mask.astype(np.uint8)
                contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                #contours = connect_closest_contours(contours, image, angle_tolerance = 5, fixed_threshold=50, pixel_threshold = 3)
                #color  = np.random.rand(3) * 255
                cv2.drawContours(image, contours, -1, color, 1)
                x1, y1, x2, y2 = bbox.astype(np.int32)
                cv2.rectangle(image, (x1, y1), (x2, y2), color, 1)
            else:  # For other labels, draw bounding boxes
                color = colors[label]
                x1, y1, x2, y2 = bbox.astype(np.int32)
                cv2.rectangle(image, (x1, y1), (x2, y2), color, 1)
    
    cv2.imwrite(os.path.join("outputall", file), image)

In [ ]:
import torch

torch.cuda.empty_cache()

In [ ]:
import tifffile

In [ ]:
img = tifffile.imread(r"C:\Users\philmarq\Downloads\dataset_seg_2ch_256_updated\dataset_seg_2ch_256_updated\valid\images\T20_CD3_CD8_whole_scene-0_base_roi-0_x-768_y-960.tif")

In [ ]:
img.shape

In [ ]:
import os
from flask import Flask, request, jsonify
import cv2
import numpy as np
from mmdet.apis import init_detector, inference_detector

app = Flask(__name__)

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

config_file = r"C:\Users\philmarq\source\repos\MMWrapperRepo\megdata\swin_s.py"
checkpoint_file = r"C:\Users\philmarq\source\repos\MMWrapperRepo\megdata\best_coco_bbox_mAP_epoch_55.pth"
device = 'cuda:0'

model = init_detector(config_file, checkpoint_file, device=device)

@app.route('/inference', methods=['POST'])
def perform_inference():
    if 'image' not in request.files:
        return jsonify({'error': 'No image uploaded'}), 400

    image_file = request.files['image']
    if image_file.filename == '':
        return jsonify({'error': 'No selected file'}), 400


    image = cv2.imdecode(np.frombuffer(image_file.read(), np.uint8), cv2.IMREAD_COLOR)
    result = inference_detector(model, image)

    masks = result[0].cpu().numpy()  
    scores = result[1].cpu().numpy()
    labels = result[2].cpu().numpy()

    response_data = {
        'message': 'Inference successful',
        'masks': masks.tolist(),
        'scores': scores.tolist(),
        'labels': labels.tolist(),
    }

    return jsonify(response_data)

if __name__ == '__main__':
    app.run(host='127.0.0.1', port=5000)


In [ ]:
import requests

url = 'http://localhost:5000/inference'

image_path = 'test.jpg'

files = {'image': open(image_path, 'rb')}

response = requests.post(url, files=files)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f'Error: {response.status_code}')
    print(response.text)
